In [ ]:
'''

'''

from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import os, sys
import copy
import imageio
import scipy.io as io
import random
import skimage
import pandas as pd
import shutil
import pickle

from glob import glob
from random import shuffle
from datetime import datetime
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPooling3D, Activation, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from sklearn.utils import class_weight
from skimage.transform import rescale, resize, downscale_local_mean

import keras.backend as K
from tqdm import tqdm


In [ ]:


fname_csv = r'open\train_df.csv'
dat_csv = pd.read_csv(fname_csv)

fname_png = dat_csv.loc[:, "file_name"]
class1 = dat_csv.loc[:, "class"]
state1 = dat_csv.loc[:, "state"]

# print(fname_png)
print(class1)

X_img = list()
Xc_img = {}
yc_class = {}
yc_state = {}

y_class = list()
y_state = list()

fname_glob = glob(r'open\train\*.png')

for kk, fname1 in tqdm(enumerate(fname_glob)):
    
    # print(f'{kk} = {fname1}')
    img = imageio.imread(fname1)
    img2 = resize(img, (256, 256), anti_aliasing=True)

    # print(img2.shape)

    if img2.ndim==2:
        #img2 = grayscale_to_rgb(img2)
        img2 = np.stack((img2,)*3, axis=-1)
    img2 = 255*img2/np.max(img2)
    img2 = img2.astype(np.uint8)    

    X_img.append(img2)
    y_class.append(class1[kk])
    y_state.append(state1[kk])

    if Xc_img.get(class1[kk]) is None:
        Xc_img[class1[kk]] = list()
        yc_class[class1[kk]] = list()
        yc_state[class1[kk]] = list()

    Xc_img[class1[kk]].append(img2)
    yc_class[class1[kk]].append(class1[kk])
    yc_state[class1[kk]].append(state1[kk])

    dir1 = os.path.join('open/train', class1[kk])

    os.makedirs(dir1, exist_ok=True)

    src = fname1
    dest = os.path.join(dir1, os.path.split(fname1)[1])
    shutil.copyfile(src, dest)


In [ ]:


data = {'Xc_img': Xc_img, 'yc_class': yc_class, 'yc_state': yc_state}
     
## 데이타 피클형식으로 저장.

with open('train_data_classwise.pickle', 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
    
    

In [ ]:
img3 = Xc_img['transistor'][0]

plt.figure()
plt.imshow(img3)
plt.show()



In [ ]:
jj = 230

img3 = X_img[jj]
print(img3.shape)

# img4 = grayscale_to_rgb(img3)

plt.figure()
plt.imshow(img3)
plt.title(f'{y_class[jj]}')
plt.show()

print(X_img[jj].shape)
print(X_img[jj].ndim)
print(img3.shape)


In [ ]:
plt.figure()
plt.imshow(X_img[917])
plt.title(f'{y_class[917]}')
plt.show()

print(img.shape)

In [ ]:
X_img = np.stack(X_img, axis=0)

print(X_img.shape)

y_class = np.stack(y_class, axis=0)

print(y_class.shape)
print(y_class[23])

print(np.unique(y_class))

class_name = np.unique(y_class)

y_num = np.zeros((y_class.shape[0]))

for k in range(class_name.shape[0]):
    print(f'{k}')
    index = np.where(y_class==class_name[k])
    y_num[index] = k
    

print(index)

print(y_class[4])

# y_num = list()

# for kk in range(y_class.shape[0]):
#     if y_class[i]==class_name[1]

In [ ]:
print(np.unique(y_num))

In [ ]:
#
# transfer learning
#

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# create the base pre-trained model
base_model = InceptionV3(weights = 'imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have 200 classes

nclass = 15
learning_rate = 0.0001

predictions = Dense(nclass, activation='softmax')(x)


# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from sklearn.model_selection import train_test_split

now = datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H%M%S")

filepath = os.path.join('open/model_check', dt_string) + '/CNN_e{epoch:02d}.h5'


checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_num), y=y_num)
class_weights = dict(enumerate(class_weights))

print(class_weights)

# train the model on the new data for a few epochs
# model.fit(...)


gpu_no = '/gpu:0'



y_num_encode = to_categorical(y_num)  # one hot encode



print(y_num_encode[1,:])
print(X_img.shape, y_num.shape, y_num_encode.shape)


X_train, X_val, y_train, y_val = train_test_split(X_img, y_num_encode, test_size=0.2, random_state=1234141)



print(X_train.shape, y_train.shape)

batch_size = 4
n_epochs = 10
ntrain = X_train.shape[0]
nvalid = X_val.shape[0]
steps_per_epoch = ntrain//batch_size


plt.figure()
plt.imshow(X_train[4,:,:,:])
plt.title(f'{y_train[4,:]}')
plt.show()

with K.tf.device(gpu_no):
    
    history = model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=n_epochs, steps_per_epoch=steps_per_epoch, 
                       class_weight=class_weights, 
                        callbacks = callbacks_list,
                        validation_data=(X_val, y_val) )

# # at this point, the top layers are well trained and we can start fine-tuning
# # convolutional layers from inception V3. We will freeze the bottom N layers
# # and train the remaining top layers.

# # let's visualize layer names and layer indices to see how many layers
# # we should freeze:
# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name)

# # we chose to train the top 2 inception blocks, i.e. we will freeze
# # the first 249 layers and unfreeze the rest:
# for layer in model.layers[:249]:
#    layer.trainable = False
# for layer in model.layers[249:]:
#    layer.trainable = True

# # we need to recompile the model for these modifications to take effect
# # we use SGD with a low learning rate
# from tensorflow.keras.optimizers import SGD

# model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# # we train our model again (this time fine-tuning the top 2 inception blocks
# # alongside the top Dense layers
# model.fit(...)


In [ ]:
# plot training and validation accuracy/loss values

print(history.history)

plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy'); plt.xlabel('Epoch')
plt.ylim([0.5, 1.0])
plt.grid(True)
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss'); plt.xlabel('epoch')
plt.grid(True)
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()